In [1]:
import os
import numpy as np
import pandas as pd
import random

'''folder_path = r"D:\\"
mmap_dir = r"C:\Users\musab\mmap_files_combined_t-3" 
'''

In [7]:

parent_dir_param = None

if parent_dir_param is None:
    parent_dir = os.path.abspath(r'../data/') 
else:
    parent_dir = os.path.abspath(parent_dir_param) 

raw_data_dir = os.path.join(parent_dir, 'raw')

processed_data_dir = os.path.join(parent_dir, 'processed_data')

base_dir_name = 'mmap_files_combined'

increment = 1
mmap_dir = os.path.join(processed_data_dir, base_dir_name)
while os.path.exists(mmap_dir):
    incremented_dir_name = f'{base_dir_name}_{increment}'
    mmap_dir = os.path.join(processed_data_dir, incremented_dir_name)
    increment += 1

os.makedirs(mmap_dir)

print(f"Using directory: '{mmap_dir}'")


Using directory: 'c:\Users\musab\2023_musa-kaleem_martin\data\processed_data\mmap_files_combined'


In [8]:
sensors = (1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 45, 53, 54, 56, 57, 58, 59, 60, 61, 62, 63, 64)
sample_size = 3




def read_int_files(folder_path, sensors, mmap_dir, sample_size):
    for folder_name in os.listdir(folder_path):
        if folder_name.startswith("Yaw"):
            folder_dir = os.path.join(folder_path, folder_name)
            files = []
            file_handles = [] 

            for file_name in os.listdir(folder_dir):
                if file_name.endswith(".int"):
                    file_path = os.path.join(folder_dir, file_name)
                    files.append(file_path)

                    file_handle = open(file_path, "rb")
                    file_handles.append(file_handle)

            dfs = [] 
            sensor_labels = []  

            for f in file_handles:
                leer = np.frombuffer(f.read(4), dtype=np.single)
                datei = np.frombuffer(f.read(24), dtype=np.uint32)
                f.seek(76)
                lvek = np.frombuffer(f.read(4), dtype=np.uint32)
                f.seek(84 + lvek[0] * 4)
                filetype = np.frombuffer(f.read(4), dtype=np.uint32)  

                if filetype != 12:
                    t0 = np.frombuffer(f.read(4), dtype=np.single)
                    dt = np.frombuffer(f.read(4), dtype=np.single)
                    pos_fak = f.tell()
                    fak = []

                    for n in sensors:
                        f.seek(pos_fak + 4 * (n - 1))
                        fak.append(np.frombuffer(f.read(4), dtype=np.single))

                    f.seek(pos_fak + lvek[0] * 4)
                    position = f.tell()

                    f.seek(0, os.SEEK_END)
                    file_size = f.tell()

                    record_count = round((file_size - position) / lvek[0] / 2)

                    sampled_indices = random.sample(range(record_count), sample_size)

                    raw_ts_int = np.empty((sample_size, len(sensors)))

                    for i, n in enumerate(sampled_indices):
                        for m, sensor_ in enumerate(sensors):
                            f.seek(position + 2 * sensor_ + (lvek[0] * 2) * n - 2)
                            raw_ts_int[i, m] = np.frombuffer(f.read(2), dtype=np.int16) * fak[m]

                    dfs.append(raw_ts_int)

                    sensor_labels.extend([f"Sensor {sensor}" for sensor in sensors])

            for f in file_handles:
                f.close()

            combined_array = np.concatenate(dfs, axis=0)

            combined_mmap_path = os.path.join(mmap_dir, f"{folder_name}_combine.mmap")
            combined_mmap = np.memmap(combined_mmap_path, dtype=np.float32, mode="w+", shape=combined_array.shape)
            combined_mmap[:] = combined_array

    return combined_mmap, sensor_labels


os.makedirs(mmap_dir, exist_ok=True)

result_mmap, sensor_labels = read_int_files(folder_path, sensors, mmap_dir, sample_size)

result_mmap = result_mmap.reshape((-1, len(sensors)))

df = pd.DataFrame(result_mmap, columns=sensor_labels[:len(sensors)])

print(df)


In [9]:


# Function to read and process int files
def read_int_files(folder_path, sensors, mmap_dir, sample_size):
    for folder_name in os.listdir(folder_path):
        if folder_name.startswith("Yaw"):
            folder_dir = os.path.join(folder_path, folder_name)
            files = []
            file_handles = []

            for file_name in os.listdir(folder_dir):
                if file_name.endswith(".int"):
                    file_path = os.path.join(folder_dir, file_name)
                    files.append(file_path)

                    file_handle = open(file_path, "rb")
                    file_handles.append(file_handle)

            dfs = []
            sensor_labels = []

            for f in file_handles:
                leer = np.frombuffer(f.read(4), dtype=np.single)
                datei = np.frombuffer(f.read(24), dtype=np.uint32)
                f.seek(76)
                lvek = np.frombuffer(f.read(4), dtype=np.uint32)
                f.seek(84 + lvek[0] * 4)
                filetype = np.frombuffer(f.read(4), dtype=np.uint32)

                if filetype != 12:
                    t0 = np.frombuffer(f.read(4), dtype=np.single)
                    dt = np.frombuffer(f.read(4), dtype=np.single)
                    pos_fak = f.tell()
                    fak = []

                    for n in sensors:
                        f.seek(pos_fak + 4 * (n - 1))
                        fak.append(np.frombuffer(f.read(4), dtype=np.single))

                    f.seek(pos_fak + lvek[0] * 4)
                    position = f.tell()

                    f.seek(0, os.SEEK_END)
                    file_size = f.tell()

                    record_count = round((file_size - position) / lvek[0] / 2)

                    sampled_indices = random.sample(range(record_count), sample_size)

                    raw_ts_int = np.empty((sample_size, len(sensors)))

                    for i, n in enumerate(sampled_indices):
                        for m, sensor_ in enumerate(sensors):
                            f.seek(position + 2 * sensor_ + (lvek[0] * 2) * n - 2)
                            raw_ts_int[i, m] = np.frombuffer(f.read(2), dtype=np.int16) * fak[m]

                    dfs.append(raw_ts_int)

                    sensor_labels.extend([f"Sensor {sensor}" for sensor in sensors])

            for f in file_handles:
                f.close()

            combined_array = np.concatenate(dfs, axis=0)

            combined_mmap_path = os.path.join(mmap_dir, f"{folder_name}_combine.mmap")
            combined_mmap = np.memmap(combined_mmap_path, dtype=np.float32, mode="w+", shape=combined_array.shape)
            combined_mmap[:] = combined_array

    return combined_mmap, sensor_labels

# Call the function with the updated paths
result_mmap, sensor_labels = read_int_files(raw_data_dir, sensors, mmap_dir, sample_size)

result_mmap = result_mmap.reshape((-1, len(sensors)))

df = pd.DataFrame(result_mmap, columns=sensor_labels[:len(sensors)])

print(df)


    Sensor 1  Sensor 4      Sensor 5      Sensor 6      Sensor 7    Sensor 8  \
0   4.307956 -5.509710 -4.426729e-14  2.317205e-12 -2.182664e-12  186.295990   
1   3.763908 -6.910822 -3.402003e-14  4.890159e-12 -4.490031e-12  146.229187   
2   4.028133 -6.506264 -7.999643e-14  4.468332e-12 -4.193695e-12   78.536324   
3   4.361075 -5.673108 -1.448766e-13  4.556320e-12 -4.416034e-12  341.329620   
4   3.682673 -4.839239 -8.702779e-14  3.920196e-12 -3.728665e-12  214.679581   
5   4.224463 -6.102754 -1.334388e-13  2.125339e-12 -2.178565e-12  295.126770   
6   4.229456  1.345344 -1.918048e-14  6.172103e-13 -6.065422e-13  140.829498   
7   3.834493 -7.658765 -4.619525e-14  4.002400e-12 -3.735823e-12  137.690948   
8   4.264898 -3.979904 -6.099103e-14  1.414282e-12 -1.408907e-12   49.226765   
9   4.211177 -6.776806 -5.138910e-14  1.087446e-12 -1.090339e-12  202.916077   
10  4.125835 -2.701869 -1.566401e-13  3.628960e-12 -3.597299e-12  328.600952   
11  3.592372 -3.156102 -7.616805e-14  1.